In [1]:
#IMPORTING ESSENTIAL LIBRARIES
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from joblib import dump

In [91]:
selected_classification = "Pattern Type"

df = pd.read_csv('dark_patterns.csv')

df.head()

,Pattern String,Comment,Pattern Category,Pattern Type,Where in website?,Deceptive?,Website Page
0,Collin P. from Grandview Missouri just bought ...,Periodic popup,Social Proof,Activity Notification,Product Page,No,https://alaindupetit.com/collections/all-suits...
1,"Faith in Glendale, United States purchased a C...",Periodic popup,Social Proof,Activity Notification,Product Page,No,https://bonescoffee.com/products/strawberry-ch...
2,Sharmeen Atif From Karachi just bought Stylish...,Periodic popup,Social Proof,Activity Notification,Product Page,No,https://brandsego.com/collections/under-rs-99/...
3,9 people are viewing this.,Product detail,Social Proof,Activity Notification,Product Page,No,https://brightechshop.com/products/ambience-so...
4,5338 people viewed this in the last hour,Periodic popup,Social Proof,Activity Notification,Product Page,No,https://bumpboxes.com/


In [92]:
df = df[pd.notnull(df["Pattern String"])]
col = ["Pattern String", selected_classification]
df = df[col]

df.head()


,Pattern String,Pattern Type
0,Collin P. from Grandview Missouri just bought ...,Activity Notification
1,"Faith in Glendale, United States purchased a C...",Activity Notification
2,Sharmeen Atif From Karachi just bought Stylish...,Activity Notification
3,9 people are viewing this.,Activity Notification
4,5338 people viewed this in the last hour,Activity Notification


In [75]:
df["category_id"] = df[selected_classification].factorize()[0]
category_id_df = df[[selected_classification, 'category_id']
                    ].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(
    category_id_df[['category_id', selected_classification]].values)

In [76]:

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2',
                        encoding='latin-1', ngram_range=(1, 2), stop_words='english')

In [77]:
X_train, X_test, y_train, y_test = train_test_split(
    df['Pattern String'], df[selected_classification], train_size=0.8)
#USING train_size 0.7

In [78]:
#Text vectorization for training set
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [79]:
clf_bayes = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf_bayes.predict(count_vect.transform(X_test))

print("Accuracy:", metrics.accuracy_score(y_pred, y_test))

Accuracy: 0.9240924092409241


In [80]:
from sklearn.svm import SVC
clf_svc = SVC(kernel = 'rbf').fit(X_train_tfidf, y_train)
y_pred = clf_svc.predict(count_vect.transform(X_test))

print("Accuracy:", metrics.accuracy_score(y_pred, y_test))

Accuracy: 0.5115511551155115


In [81]:
from sklearn.tree import DecisionTreeClassifier
clf_decTree = DecisionTreeClassifier(criterion = 'entropy', random_state = 0).fit(X_train_tfidf, y_train)
y_pred = clf_decTree.predict(count_vect.transform(X_test))

print("Accuracy:", metrics.accuracy_score(y_pred, y_test))

Accuracy: 0.8151815181518152


In [82]:
from sklearn.ensemble import RandomForestClassifier
clf_randomForest = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0).fit(X_train_tfidf, y_train)
y_pred = clf_randomForest.predict(count_vect.transform(X_test))

print("Accuracy:", metrics.accuracy_score(y_pred, y_test))

Accuracy: 0.9174917491749175
